# Compute distance between markup points

Import modules and define functions

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from io import StringIO

# numpy disable scientific notation for easier debugging
np.set_printoptions(suppress=True, precision=4)

# make prototype for storing CT, T1 and electrodes filenames
class SubjectFiles:
    def __init__(self, subject_root, ct_file, t1_file, electrodes_file):
        self.subject_root = subject_root
        self.ct_file = ct_file
        self.t1_file = t1_file
        self.electrodes_file = electrodes_file

    def ct_date(self):
        return self.get_date(self.ct_file)
    
    def t1_date(self):
        return self.get_date(self.t1_file)

    @staticmethod
    def get_date(filename):
        match = re.search(r"_(\d{8})_", filename)
        if match:
            return match.group(1)
        return None
    
def load_json(path: str) -> pd.DataFrame:
    markups_prediction_json = json.load(open(path, "r"))
    assert markups_prediction_json["markups"][0]["coordinateUnits"] == "mm"
    assert markups_prediction_json["markups"][0]["coordinateSystem"] == "LPS"
    df = pd.DataFrame(markups_prediction_json["markups"][0]["controlPoints"], columns=["label", "position"])
    df["position"] = df["position"].apply(lambda x: [-x[0], -x[1], x[2]]) # LPS -> RAS
    return df

def load_fcsv(path: str) -> pd.DataFrame:
    with open(path, "r") as f:
        electrodes_str = f.read().replace(",,", ",")
    for line in electrodes_str.split("\n"):
        # get header
        if line.startswith("# columns = "):
            header_line = line.strip().replace("# columns = ", "")
            columns = header_line.split(",")
            break

    # read lines
    electrodes_df = pd.read_csv(StringIO(electrodes_str), sep=",", skiprows=3, names=columns)
    electrodes_df["position"] = electrodes_df[["x", "y", "z"]].apply(lambda x: list(x), axis=1)
    return electrodes_df[["label", "position"]]


Load data

In [8]:
input_dir = "../../Data"
subjects: list[SubjectFiles] = []

# walk through all subfolders and search for *CT*, *T1* and electrodes.fcsv files
for root, dirs, files in os.walk(input_dir):
    ct_path = None
    t1_path = None
    fcsv_path = None

    for file in files:
        if "CT" in file and file.endswith((".nii", ".nii.gz")):
            ct_path = file
        elif "T1" in file and not file.startswith("rand_affine_") and file.endswith((".nii", ".nii.gz")):
            t1_path = file
        elif file == "ContactDetector.mrk.json":
            fcsv_path = file
    if ct_path and t1_path and fcsv_path:
        root = root.replace(input_dir + os.sep, "")
        subjects.append(SubjectFiles(root, ct_path, t1_path, fcsv_path))

Compute distance between markup points:

In [9]:
markups_all = []
for subject in subjects:
    gt_df = load_fcsv(os.path.join(input_dir, subject.subject_root, "electrodes.fcsv"))
    contact_detector_df = load_json(os.path.join(input_dir, subject.subject_root, "ContactDetector.mrk.json"))

    # merge gt and contact detector df on label
    markups = pd.merge(gt_df, contact_detector_df, on="label", how="outer", suffixes=("_gt", "_contact_detector"))
    markups["prefix"] = markups["label"].str.extract(r"(.*?)(\d+)$")[0]
    markups["contact"] = markups["label"].str.extract(r"(.*?)(\d+)$")[1]
    markups["subject_root"] = subject.subject_root
    # change column order
    markups = markups[["subject_root", "label", "prefix", "contact", "position_gt", "position_contact_detector"]]

    for index, row in markups.iterrows():
        markups.loc[index, "norm(gt,contact detector)"] = np.linalg.norm(np.array(row["position_gt"]) - np.array(row["position_contact_detector"]))

    markups_all.append(markups)

markups = pd.concat(markups_all)

# save to csv
markups.to_csv("results.csv", index=False)

markups

,subject_root,label,prefix,contact,position_gt,position_contact_detector,"norm(gt,contact detector)"
0,1673284,A1,A,1,"[0.05, 29.23, 53.48]","[0.1950078424448236, 29.200404812578512, 53.48...",0.148224
1,1673284,A10,A,10,"[25.37, 40.43, 68.49]","[25.484877695550736, 40.4610317433888, 68.5283...",0.125020
2,1673284,A11,A,11,"[28.13, 41.62, 70.28]","[28.210731466782633, 41.6489976962098, 70.3125...",0.091751
3,1673284,A12,A,12,"[30.89, 42.77, 72.12]","[30.991657464917544, 42.82190185332607, 72.189...",0.133810
4,1673284,A2,A,2,"[2.93, 30.4, 55.11]","[3.011295099902938, 30.451242471122328, 55.152...",0.105150
...,...,...,...,...,...,...,...
192,890775,Xs5,Xs,5,"[-31.73, 15.22, 24.4]","[-31.708021862584374, 15.310408906482692, 24.3...",0.094471
193,890775,Xs6,Xs,6,"[-31.76, 14.41, 27.93]","[-31.727884325011175, 14.5275756956121, 27.788...",0.186692
194,890775,Xs7,Xs,7,"[-31.78, 13.6, 31.46]","[-31.751128980735515, 13.745345189645349, 31.1...",0.304568
195,890775,Xs8,Xs,8,"[-31.8, 12.79, 34.98]","[-31.777848477621674, 12.939084800403819, 34.6...",0.321474


Table of missing contact detections:

In [10]:
markups[markups["position_contact_detector"].isna() | markups["position_gt"].isna()]

,subject_root,label,prefix,contact,position_gt,position_contact_detector,"norm(gt,contact detector)"
71,1673284,F10,F,10,"[8.37, 38.25, 99.87]",NaN,NaN
77,1673284,F7,F,7,"[4.23, 32.45, 92.11]",NaN,NaN
78,1673284,F8,F,8,"[5.64, 34.38, 94.67]",NaN,NaN
79,1673284,F9,F,9,"[7.02, 36.3, 97.25]",NaN,NaN
17,1219229/i2,K10,K,10,"[-28.79, -13.23, 63.05]",NaN,NaN
177,2061851,N11,N,11,"[35.47, -53.35, 7.68]",NaN,NaN
178,2061851,N12,N,12,"[38.91, -53.79, 7.19]",NaN,NaN
179,2061851,N13,N,13,"[42.35, -54.23, 6.71]",NaN,NaN
180,2061851,N14,N,14,"[45.79, -54.67, 6.24]",NaN,NaN
181,2061851,N15,N,15,"[49.22, -55.1, 5.77]",NaN,NaN


Group by subject_root and prefix, get max norm and mean norm for each group:

In [11]:
electrodes = markups.groupby(["subject_root", "prefix"]).agg({
    "subject_root": "first",
    "norm(gt,contact detector)": ["max", "mean"]
}).sort_values(("norm(gt,contact detector)", "max"), ascending=False)
electrodes

subject_root norm(gt,contact detector)           
                           first                       max       mean
subject_root prefix                                                  
2394232      I           2394232                 59.522406  51.560652
             H           2394232                 58.248929  50.112368
2061851      N           2061851                 53.780912  49.415620
1673284      F           1673284                 47.822008  38.238682
2061851      L           2061851                 40.382081  30.350048
...                          ...                       ...        ...
1085916      Z           1085916                  0.053517   0.028889
37708        SM            37708                  0.052323   0.030695
1096731/i2   J        1096731/i2                  0.051900   0.031688
1410956      B           1410956                  0.049997   0.030105
366954       CM           366954                  0.049956   0.032555

[1153 rows x 3 columns]

Consider mean norm > 1 as failed detection for particular electrode:

In [12]:
# for subject_root count norm.mean > 1
failed = electrodes[electrodes[("norm(gt,contact detector)", "mean")] > 1]
# sort by ('subject_root', 'first') and ('norm(gt,contact detector)', 'max')
failed = failed.sort_values([("subject_root", "first"), ("norm(gt,contact detector)", "max")])
failed

subject_root norm(gt,contact detector)           
                           first                       max       mean
subject_root prefix                                                  
1096731/i1   F        1096731/i1                  3.573589   3.549966
1157846      J           1157846                  3.815293   3.750278
             L           1157846                  3.963573   3.567123
1219229/i2   O        1219229/i2                  3.764858   3.337226
             J        1219229/i2                  4.626916   3.851052
...                          ...                       ...        ...
40259        M             40259                  3.665325   3.641709
             L             40259                  3.734306   3.704652
890775       Bd           890775                 12.844238  10.444255
             Ed           890775                 15.736457  13.667589
             Cd           890775                 28.285537  28.172552

[67 rows x 3 columns]